# Self-Driving Car Engineer Nanodegree Term1
## Project 4 : Advanced Lane Finding
---

In this project, your goal is to write a software pipeline to identify the lane boundaries in a video from a front-facing camera on a car. The camera calibration images, test road images, and project videos are available in the [project repository](https://github.com/udacity/CarND-Advanced-Lane-Lines).

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Advanced-Lane-Lines/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/571/view) for this project.


The method of computing the calibration and undistortion is in [this repository](https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb).

https://discussions.udacity.com/t/smoothing-the-line/352850/10

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import re
import pickle

from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

## Camera Calibration

### Extract object points and image points for camera calibration.

In [2]:
with open("wide_dist_pickle.p", 'rb') as fp:
    dist_pickle = pickle.load(fp)
    
mtx, dist = dist_pickle["mtx"], dist_pickle["dist"]

## Single Image (pipeline)

### Helper functions

In [3]:
def undistort_image(image):
    """
    image is a BGR color image
        
    Returns an undistored img RGB color iamge
    """
    # Getting image size
    img_size = (image.shape[0], image.shape[1])
    
    # Converting to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Undisorting image
    undst = cv2.undistort(image, mtx, dist, None, mtx)
    
    # Convert BGR to RGB
    img = cv2.cvtColor(undst, cv2.COLOR_BGR2RGB)
    
    return img

# should produce output like the example image shown above this quiz.
def abs_sobel_thresh(img, orient='x', sobel_kernel = 3, thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # isn't mpimg.imread() should use BGR2GRAY to convert to gray?
    
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient=='x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel>=thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    
    # 6) Return this mask as your binary_output image
    return binary_output


# Define a function that applies Sobel x and y, 
# then computes the magnitude of the gradient
# and applies a threshold
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0 ,1, ksize=sobel_kernel))
    
    # 3) Calculate the magnitude 
    magnitude = np.sqrt(sobelx**2 + sobely**2)
    
    
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255*magnitude/np.max(magnitude))
    
    # 5) Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    
    # 6) Return this mask as your binary_output image
    return binary_output


# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    directions = np.arctan2(abs_sobely, abs_sobelx)
    
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(directions)
    binary_output[(directions>=thresh[0]) & (directions < thresh[1])] = 1 # Remove this line
    
    # 6) Return this mask as your binary_output image
    return binary_output


def color_threshold(img):
    
    # Apply the following steps to img
    # 1) Convert to HLS: hue, lightness, saturation
    #hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    #s_channel = hls[:,:,2]
    #l_channel = hls[:,:,1]
    #h_channel = hls[:,:,0]
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    l_channel = luv[:,:,0]  #lightness
    
    #hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #s_channel = hsv[:,:,1]  #saturation
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    
    
    # 2) Create a binary mask where color channel thresholds are met
    binary_output = np.zeros_like(s_channel)
    
    '''
    # 3） Get mean saturation value
    lower_half_s_channel = s_channel[round(s_channel.shape[0]/2):, ]
    lower_half_l_channel = l_channel[round(l_channel.shape[0]/2):, ]
    #lower_half_h_channel = h_channel[round(h_channel.shape[0]/2):, ]
    if plot_flag:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
        ax1.imshow(lower_half_s_channel)
        ax2.imshow(lower_half_l_channel)
    
    s_vector = np.reshape(lower_half_s_channel, (np.product(lower_half_s_channel.shape),))
    l_vector = np.reshape(lower_half_l_channel, (np.product(lower_half_l_channel.shape),))
    #h_vector = np.reshape(lower_half_h_channel, (np.product(lower_half_h_channel.shape),))
    s_mean = np.mean(s_vector)
    s_std = np.std(s_vector)
    s_min = np.min(s_vector)
    s_max = np.max(s_vector)
    s_80 = np.percentile(s_vector, 80)
    #print('s_mean:'+str(s_mean))
    #print('s_std:'+str(s_std))
    
    l_mean = np.mean(l_vector)
    l_std = np.std(l_vector)
    l_min = np.min(l_vector)
    l_max = np.max(l_vector)
    l_80 = np.percentile(l_vector, 90)
    '''
     
    # 3) Reuturn this mask as your binary_output image
    #binary_output[(s_channel >= thresh_s) & (s_channel <= thresh_s[1]) & (l_channel >= thresh_l[0])] = 1
    #binary_output[(s_channel >= thresh_s) & (s_channel <= thresh_s[1]) & (l_channel >= thresh_l[0])] = 1
    
    thr_s, thr_l = thr1[0], thr1[1]
    thr_sum = sum_thr
    binary_output[(s_channel >= thr_s) & (l_channel >= thr_l)] = 1
    
    binary_warped = transform_perspective(binary_output, binary_output)[2]
    binary_sum = np.sum(np.sum(np.reshape(binary_warped, np.product(binary_warped.shape), 1)))
    if plot_flag:
        print('old binary_sum:'+str(binary_sum))
    
    if binary_sum > thr_sum:
        thr_s, thr_l = thr2[0], thr2[1]
        binary_output = np.zeros_like(s_channel)
        binary_output [(s_channel >= thr_s)&(l_channel >= thr_l)] = 1
        binary_warped = transform_perspective(binary_output, binary_output)[2]
        binary_sum = np.sum(np.sum(np.reshape(binary_warped, np.product(binary_warped.shape), 1)))
    if binary_sum > thr_sum:
        thr_s, thr_l = thr3[0], thr3[1]
        binary_output = np.zeros_like(s_channel)
        binary_output [(s_channel >= thr_s)&(l_channel >= thr_l)] = 1
        binary_warped = transform_perspective(binary_output, binary_output)[2]
        binary_sum = np.sum(np.sum(np.reshape(binary_warped, np.product(binary_warped.shape), 1)))
    if binary_sum > thr_sum:
        thr_s, thr_l = thr4[0], thr4[1]
        binary_output = np.zeros_like(s_channel)
        binary_output [(s_channel >= thr_s)&(l_channel >= thr_l)] = 1
        binary_warped = transform_perspective(binary_output, binary_output)[2]
        binary_sum = np.sum(np.sum(np.reshape(binary_warped, np.product(binary_warped.shape), 1)))
     
    if plot_flag:
        #print('s_mean:'+str(s_mean))
        #print('s_std:'+str(s_std))
        #print('s_min:'+str(s_min))
        #print('s_max:'+str(s_max))
        #print('s_80:'+str(s_80))
        #print('l_mean:'+str(l_mean))
        #print('l_std:'+str(l_std))
        #print('l_min:'+str(l_min))
        #print('l_max:'+str(l_max))
        #print('l_80:'+str(l_80))
        print('thresh_s:' + str(thr_s))
        print('thresh_l:' + str(thr_l))
        print('binary_sum:'+str(binary_sum))
        
        s_output, l_output = np.zeros_like(s_channel), np.zeros_like(l_channel)
        s_output[s_channel >= thr_s]=1
        l_output[l_channel >= thr_l]=1
        
    
        f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 5))
        ax1.imshow(s_channel)
        ax2.imshow(l_channel)
        ax3.imshow(s_output, cmap='gray')
        ax4.imshow(l_output, cmap='gray')
    
    
    return binary_output


def binarize_image(image):
    """
    image is an undistorted RGB color image
        
    Returns an binary undistored gray image
    """
    # Choose a Sobel kernel size
    ksize = 3 # Choose a larger odd number to smooth gradient measurements
    
    # Apply each of the thresholding functions
    #gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    #grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    #mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(30, 100))
    #dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0, np.pi/4))
    color_binary = color_threshold(image)
    
    # Binary image
    #combined = np.zeros_like(dir_binary)
    #combined[(((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))) | (color_binary == 1)] = 1
    
    combined = color_binary
    
    return combined


def transform_perspective(image, image_color):
    """
    image is an undistorted gray image
        
    Returns perspective transform M and inverse transform Minv, and warped image
    """
    # image size
    img_size = (image.shape[1], image.shape[0])

    # Specify source points:
    src_lt = [(img_size[0] / 2) - 62, img_size[1] / 2 + 100]
    src_lb = [((img_size[0] / 6) - 20), img_size[1]]
    src_rb = [(img_size[0] * 4 / 6) + 272, img_size[1]]
    src_rt = [(img_size[0] / 2 + 65), img_size[1] / 2 + 100]
    src = np.float32([src_lt, src_lb, src_rb, src_rt])
    #print('Source points: \n', src)

    # Sepcify destination points:
    dst_lt = [(img_size[0]/4), 0]
    dst_lb = [(img_size[0]/4), img_size[1]-5]
    dst_rb = [(img_size[0]*3/4), img_size[1]-5]
    dst_rt = [(img_size[0]*3/4), 0]
    dst = np.float32([dst_lt, dst_lb, dst_rb, dst_rt])
    #print('Destination points: \n', dst)

    # Compute the perspective transform, M, given source and destination points:
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)

    #Warp an image using the perspective transform, M:
    binary_warped = cv2.warpPerspective(image, M, img_size, flags=cv2.INTER_LINEAR)
    color_warped = cv2.warpPerspective(image_color, M, img_size, flags=cv2.INTER_LINEAR)
    
    return M, Minv, binary_warped, color_warped

def region_of_interest(img, ):
    """
    Applies an image mask.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img) 
    
    #filling region of interest
    mask[:, 220:600]= 1
    mask[:, 800:1150]= 1
    
    #if plot_flag:
    #    plt.imshow(mask, cmap = 'gray')
    
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image
    

def identify_lanepixels(binary_warped):
    
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[np.int32(binary_warped.shape[0]/2):,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 10
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = window_margin #100
    # Set minimum number of pixels found to recenter window
    minpix = pixel_min
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 4) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 4) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    return leftx, lefty, rightx, righty, left_lane_inds, right_lane_inds

def fit_poly(leftx, lefty, rightx, righty):
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit


def sanity_check(binary_undst_warped, left_fit, right_fit, leftx, lefty, rightx, righty, left_lane_inds, right_lane_inds, min_thr =600, max_thr=650):
    """
    Check fitted lines, if invalid, use previous valid frames
        
    Returns 
    """
    global left_fit_hist, right_fit_hist, left_fit_cur, right_fit_cur
    global leftx_hist, lefty_hist, leftx_cur, lefty_cur
    global rightx_hist, righty_hist, rightx_cur, righty_cur
    global left_lane_inds_hist, right_lane_inds_hist, left_lane_inds_cur, right_lane_inds_cur
    
    # Generate x and y for fitting
    ploty = np.linspace(0, binary_undst_warped.shape[0]-1, binary_undst_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Check max and min interval between left and right lanes
    left_right_diff = np.abs(left_fitx - right_fitx)
    min_diff, max_diff = np.min(left_right_diff), np.max(left_right_diff)
    if plot_flag:
        print('min_diff:' + str(min_diff))
        print('max_diff:' + str(max_diff))
    
    if (min_diff > min_thr)&(max_diff < max_thr):
        # update fitting function   
        if len(left_fit_hist) == 2:
            # only pop for later frames
            #print('left_fit_hist type')
            #print(type(left_fit_hist))
            left_fit_hist.pop(0)
            right_fit_hist.pop(0)   
        
        left_fit_hist.append(left_fit)
        right_fit_hist.append(right_fit)
        
        # update left and right x and y
        leftx_hist = leftx
        lefty_hist = lefty
        rightx_hist = rightx
        righty_hist = righty
        
        leftx_cur = leftx
        lefty_cur = lefty
        rightx_cur = rightx
        righty_cur = righty
        
        # update left and right inds
        left_lane_inds_hist = left_lane_inds
        right_lane_inds_hist = right_lane_inds
        
        left_lane_inds_cur = left_lane_inds
        right_lane_inds_cur = right_lane_inds
    else:
        print('Discard this frame!')
        # use previous value
        leftx_cur = leftx_hist
        lefty_cur = lefty_hist
        rightx_cur = rightx_hist
        righty_cur = righty_hist
        left_lane_inds_cur = left_lane_inds_hist
        right_lane_inds_cur = right_lane_inds_hist
         
    # Weighted average fitting
    if len(left_fit_hist) == 1: 
        # first frame
        left_fit_cur = left_fit_hist[0]
        right_fit_cur = right_fit_hist[0]
    elif len(left_fit_hist)==2:
        # successive frame
        alpha = 0.5
        left_fit_cur = (left_fit_hist[0]*alpha + left_fit_hist[1]*(1-alpha)).tolist()
        right_fit_cur = (right_fit_hist[0]*alpha + right_fit_hist[1]*(1-alpha)).tolist()
        
    #print(type(left_fit_cur))

        
def calculate_curvature(binary_warped, left_fit, right_fit, leftx, lefty, rightx, righty):
    # image size
    img_size = (binary_warped.shape[1], binary_warped.shape[0])
    
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    y_eval = np.max(ploty)
    #left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    #right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    #print(round(left_curverad, 2), round(right_curverad, 2))
    # Example values: 1926.74 1908.48

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

    # Calculate the new radius of curvature
    left_curverad = np.int(((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0]))
    right_curverad = np.int(((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0]))
    # Now our radius of curvature is in meters
    #print("Radius of curvature: ", left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m

    # Calculate the position of the vehicle with respect to center
    car_center_pix = img_size[0]/2
    road_center_pix = leftx[0] + (rightx[0] - leftx[0])/2
    diff_center_pix = car_center_pix - road_center_pix
    diff_center_m = round(diff_center_pix * xm_per_pix,2)
    #print("Position of the vehicle with respect to center: ", diff_center_m, "m")
    
    return left_curverad, right_curverad, diff_center_m

    
def plot_identified_lane(image, binary_undst_warped, Minv, left_fit, right_fit,left_curverad, right_curverad, diff_center_m):
    # Generate x and y for plotting
    ploty = np.linspace(0, binary_undst_warped.shape[0]-1, binary_undst_warped.shape[0])  
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_undst_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right)).astype(np.int32)

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    
    # Add text of curvature and vehicle distance to center
    dir_string = ['m left of center', 'm right of center']
    if right_curverad > left_curverad:
        rad = left_curverad
    else:
        rad = right_curverad
    rad_string = 'Radius of Curvature: '+ str(rad) + ' m'
    center_string = 'Vehicle is '+ str(np.abs(diff_center_m)) + ' ' + dir_string[diff_center_m > 0]
    cv2.putText(result, rad_string, (295, 100), cv2.FONT_HERSHEY_COMPLEX_SMALL , 2, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(result, center_string, (245, 200), cv2.FONT_HERSHEY_COMPLEX_SMALL , 2, (255, 255, 255), 2, cv2.LINE_AA)
    
    return result

print('Updated')

Updated


In [4]:
def process_image(image):
    global left_fit_hist, right_fit_hist, left_fit_cur, right_fit_cur
    global leftx_hist, lefty_hist, leftx_cur, lefty_cur
    global rightx_hist, righty_hist, rightx_cur, righty_cur
    global left_lane_inds_hist, right_lane_inds_hist, left_lane_inds_cur, right_lane_inds_cur
    
    
    # Getting image size
    img_size = (image.shape[1], image.shape[0])
    
    # Undistorting image
    undst = undistort_image(image)
    
    # Thresholding image
    binary_undst = binarize_image(undst)
    # 60
    
    # Transforming perspective
    M, Minv, binary_undst_warped, color_undst_warped = transform_perspective(binary_undst, undst)
       
    # Masking warped images
    binary_undst_warped = region_of_interest(binary_undst_warped)
    
    # Identifying lane line pixels
    leftx, lefty, rightx, righty, left_lane_inds, right_lane_inds = identify_lanepixels(binary_undst_warped)

    # Fitting polynomial
    left_fit, right_fit = fit_poly(leftx, lefty, rightx, righty)
    
    # Sanity check
    sanity_check(binary_undst_warped, left_fit, right_fit, leftx, lefty, rightx, righty, \
                 left_lane_inds, right_lane_inds, min_thr = thr_min, max_thr= thr_max)
    
    
    # Calculating curvature
    left_curverad, right_curverad, diff_center_m = calculate_curvature(binary_undst_warped, \
                                                                       left_fit_cur, right_fit_cur, \
                                                                       leftx_cur, lefty_cur, rightx_cur, righty_cur)
    #print('left_fit_cur')
    #print(left_fit_cur)
    
    # Plotting identified lane area
    result = plot_identified_lane(image, binary_undst_warped, Minv, left_fit_cur, right_fit_cur,
                                 left_curverad, right_curverad, diff_center_m)
    
    global plot_flag
    
    if plot_flag:
        ###########################################################################
        #### debugging
        # Generate x and y values for plotting
        
        ploty = np.linspace(0, binary_undst_warped.shape[0]-1, binary_undst_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        out_img = color_undst_warped
        # print(color_undst_warped.shape)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_undst_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        
        
        ##cv2.imwrite('test_out_img.jpg', out_img)
        ##rec_img = cv2.imread('test_out_img.jpg')
        
        
        #### debugging 
        #test_binary = 'test_'+str(count)+'.jpg'
        #test_binary_warped = 'test_warped'+str(count)+'.jpg'
        #cv2.imwrite(test_binary, binary_undst)
        #cv2.imwrite(test_bbinary_warped, binary_undst_warped)
        
        f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(40, 10))
        ax1.imshow(binary_undst, cmap='gray')
        ax2.imshow(binary_undst_warped, cmap='gray')
        ax3.imshow(out_img)
        ax3.plot(left_fitx, ploty, color='yellow', linewidth=2)
        ax3.plot(right_fitx, ploty, color='yellow', linewidth=2)
        ax4.imshow(result)
        plt.show()
        
         
        
        ####
        ###########################################################################
        
    
    # Returning processed image
    return result

# Test on Videos

(1) dark road, light lines
s_mean 31.7  32.5
s_std  35.7  37.7
s_80   50    50
l_mean 83    84.9
l_std  34    36.8
l_80   97    100
thr_s  12    62.7
l_s    135   135

(2) white road, light lanes
s_mean 35.4 not work     32.0 work 61  30.8 not work       33.4
s_std  36.16             33.0          32.6                35.7
s_80   46                37            37                  40
l_mean 107               113           110                 93.6
l_std  47.2              49            48                  48.4
l_80   157               154           151                 147
thr_s  35.4              32            12.35               62

In [5]:
video_output = 'project_output.mp4'

# (0, 0.5) (21, 22) (23, 25) (39.5, 41) (49, 50)

# declare global variables
left_fit_hist, right_fit_hist, left_fit_cur, right_fit_cur = [], [], [], []
leftx_hist, lefty_hist, leftx_cur, lefty_cur = [], [], [], []
rightx_hist, righty_hist, rightx_cur, righty_cur = [], [], [], []
left_lane_inds_hist, right_lane_inds_hist, left_lane_inds_cur, right_lane_inds_cur = [], [], [], []

thr1 = (20, 145) # 20 165
thr2 = (35, 145) # 40 145
thr3 = (60, 125)
thr4 = (80, 125)
thr_min = 495  # 577
thr_max = 855  # 796
sum_thr = 80000
pixel_min = 40
window_margin = 100

#(6, 8) (23 25) (42 43)
# max: 680, 684, 683, 695, 676, 660, 658, 673, 721

# 854 max

plot_flag = False
# process video images
#clip1 = VideoFileClip("test.mp4").subclip(22.8, 23.5) # 
clip1 = VideoFileClip("test.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(video_output, audio=False)

# binary_sum 62227 (25 threshold)
# max_diff 729.14  < 700     60, 145

[MoviePy] >>>> Building video project_output.mp4
[MoviePy] Writing video project_output.mp4


 48%|████▊     | 611/1261 [01:01<01:13,  8.84it/s]

Discard this frame!


100%|█████████▉| 1260/1261 [02:03<00:00, 11.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_output.mp4 

CPU times: user 4min 46s, sys: 18.2 s, total: 5min 4s
Wall time: 2min 4s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [6]:
HTML("""
<video width="489" height="270" controls>
  <source src="{0}">
</video>
""".format(video_output))

https://youtu.be/etul2D4iIOU